<a href="https://colab.research.google.com/github/sivask01/CMPE-255-01/blob/main/face_mask_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# from PIL import Image
# import xml.etree.ElementTree as ET

In [ ]:
#For model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
#For shwing images and graphs
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
extracted_folder_path = '/content/drive/MyDrive/Colab Notebooks/255_Project/'
extracted_folder_path

'/content/drive/MyDrive/Colab Notebooks/255_Project/'

#**Parellel Processing**

In [24]:
# # Example of using MirroredStrategy for multi-GPU training
# import tensorflow as tf

# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
#     # Reinitialize and compile your model here


#**DATA LOADING**

In [11]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Function to display images
def display_images(folder_path, num_images=3):
    image_files = os.listdir(folder_path)[:num_images]
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

    for ax, image_file in zip(axes, image_files):
        image_path = os.path.join(folder_path, image_file)
        img = mpimg.imread(image_path)
        ax.imshow(img)
        ax.axis('off')
    plt.show()

# Displaying images from each category
print("Images from CMFD (Correctly Masked Face Dataset):")
# display_images(f"{extracted_folder_path}/Classes/CMFD")

print("Images from IMFD (Incorrectly Masked Face Dataset):")
# display_images(f"{extracted_folder_path}/Classes/IMFD")

Images from CMFD (Correctly Masked Face Dataset):
Images from IMFD (Incorrectly Masked Face Dataset):


#**DATA PRE PROCESSING**

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

# Parameters for preprocessing
image_size = (64, 64)  # Standard size for the images
batch_size = 16        # Batch size for processing

# Setting up the ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,       # Normalizing the pixel values
    validation_split=0.2  # Splitting 20% data for validation
)

# Preparing the training and validation generators
train_generator = datagen.flow_from_directory(
    directory=f"{extracted_folder_path}/Classes",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',   # Since we have two classes
    subset='training'      # Training data
)

validation_generator = datagen.flow_from_directory(
    directory=f"{extracted_folder_path}/Classes",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',   # Since we have two classes
    subset='validation'    # Validation data
)

# Checking the class indices
class_indices = train_generator.class_indices
class_indices


Found 1503 images belonging to 2 classes.
Found 375 images belonging to 2 classes.


{'CMFD': 0, 'IMFD': 1}

#**MODEL ARCHITECTURE**

##**Creating a Model(CNN)**

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])


##**Compile & Train the Model**

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=2 # Adjust as necessary
)

Epoch 1/2
93/93 [==============================] - 36s 375ms/step - loss: 0.3552 - accuracy: 0.8393 - val_loss: 0.0876 - val_accuracy: 0.9701
Epoch 2/2
93/93 [==============================] - 40s 435ms/step - loss: 0.0927 - accuracy: 0.9684 - val_loss: 0.0571 - val_accuracy: 0.9837


#**Model Testing**

In [26]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Path to your test folder
test_folder_path = f"{extracted_folder_path}/Test_Classes"

# Get all image paths from the test folder
test_images = [os.path.join(test_folder_path, file) for file in os.listdir(test_folder_path) if file.endswith('.jpg')]

# Function to preprocess and predict for a single image
def predict_image(image_path, model):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(64, 64))
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict
    prediction = model.predict(img_array)
    return prediction

# Predict each test image
for image_path in test_images:
    prediction = predict_image(image_path, model)
    class_prediction = 'CMFD' if prediction[0][0] < 0.5 else 'IMFD'
    print(f"Image: {os.path.basename(image_path)}, Prediction: {class_prediction}")


1/1 [==============================] - 0s 115ms/step
Image: 37002_Mask.jpg, Prediction: CMFD
1/1 [==============================] - 0s 29ms/step
Image: 37003_Mask.jpg, Prediction: CMFD
1/1 [==============================] - 0s 28ms/step
Image: 37001_Mask.jpg, Prediction: CMFD
